In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/prediction/custom_prediction_routines/SDK_Custom_Predict_SDK_Integration.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/prediction/custom_prediction_routines/SDK_Custom_Predict_SDK_Integration.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This tutorial demonstrates how to use Vertex AI SDK to build a custom container that uses the Custom Prediction Routine model server to serve a scikit-learn model on Vertex AI Predictions. This is currently a **preview** feature. Pre-GA products and features might have limited support, and changes to pre-GA products and features might not be compatible with other pre-GA versions. For more information, see the [launch stage descriptions](https://cloud.google.com/products#product-launch-stages).



### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small dataset that is popular for trying out machine learning techniques. Each instance has four numerical features, which are different measurements of a flower, and a target label that
marks it as one of three types of iris: Iris setosa, Iris versicolour, or Iris virginica.

This tutorial uses [the copy of the Iris dataset included in the
scikit-learn library](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html).

### Objective

The goal is to:
- Train a model that uses a flower's measurements as input to predict what type of iris it is.
- Save the model and its serialized pre-processor
- Build a custom sklearn serving container with custom prediction using the Custom Prediction Routine feature in the Vertex AI SDK
- Test the built container locally
- Upload and deploy custom container to Vertex Prediction

This tutorial focuses more on deploying this model with Vertex AI than on
the design of the model itself.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* Docker
* Git
* Google Cloud SDK (gcloud)
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as NumPy, Scikit-learn, FastAPI, Uvicorn, and joblib.

In [ ]:
%%writefile requirements.txt
fastapi
uvicorn==0.17.6
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0

**The model you deploy will have a different set of dependencies pre-installed than your notebook environment has. You should not assume that because things work in the notebook, they will work in the model. Instead, you will be very explicit about the dependencies for the model by listing them in requirements.txt and then use `pip install` to install the exact same dependencies in the notebook. Please note, of course, that there is a chance that you miss a dependency in requirements.txt that already exists in the notebook. If that's the case, things will run in the notebook, but not in the model. To guard against that, you will test the model locally before deploying to the cloud.**

In [ ]:
# Install the same dependencies used in the serving container in the notebook
# environment.
%pip install -U --user -r requirements.txt

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up logging in this sample

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` or `%` as shell commands, and it interpolates Python variables with `$` or `{}` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

### Configure project and resource names

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

Configure GCP resource names.

In [ ]:
MODEL_ARTIFACT_DIR = "sklearn-cpr-model-sdk"  # @param {type:"string"}
REPOSITORY = "custom-container-prediction-sdk"  # @param {type:"string"}
IMAGE = "sklearn-cpr-server-sdk"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "sklearn-cpr-model-sdk"  # @param {type:"string"}

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that will be pushed.

`MODEL_DISPLAY_NAME` - Display name of Vertex AI Model resource.

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

To update your model artifacts without re-building the container, you must upload your model
artifacts and any custom code to Cloud Storage.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

## Write your pre-processor

Set the directory to put your all custom code.

In [ ]:
USER_SRC_DIR = "src_dir_sdk"  # @param {type:"string"}

Decide the directory to put your trained models.

In [ ]:
LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts"  # @param {type:"string"}

Scaling training data so each numerical feature column has a mean of 0 and a standard deviation of 1 [can improve your model](https://developers.google.com/machine-learning/crash-course/representation/cleaning-data).

Create `preprocess.py`, which contains a class to do this scaling:

In [ ]:
%mkdir $USER_SRC_DIR
%mkdir $LOCAL_MODEL_ARTIFACTS_DIR

In [ ]:
%%writefile $USER_SRC_DIR/preprocess.py
import numpy as np

class MySimpleScaler(object):
    def __init__(self):
        self._means = None
        self._stds = None

    def preprocess(self, data):
        if self._means is None:  # during training only
            self._means = np.mean(data, axis=0)

        if self._stds is None:  # during training only
            self._stds = np.std(data, axis=0)
            if not self._stds.all():
                raise ValueError("At least one column has standard deviation of 0.")

        return (data - self._means) / self._stds

## Train and store model with pre-processor
Next, use `preprocess.MySimpleScaler` to preprocess the iris data, then train a model using scikit-learn.

At the end, export your trained model as a joblib (`.joblib`) file and export your `MySimpleScaler` instance as a pickle (`.pkl`) file:

In [ ]:
%cd $USER_SRC_DIR/

import pickle

import joblib
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
scaler = MySimpleScaler()

X = scaler.preprocess(iris.data)
y = iris.target

model = RandomForestClassifier()
model.fit(X, y)

joblib.dump(model, f"../{LOCAL_MODEL_ARTIFACTS_DIR}/model.joblib")
with open(f"../{LOCAL_MODEL_ARTIFACTS_DIR}/preprocessor.pkl", "wb") as f:
    pickle.dump(scaler, f)

### Upload model artifacts and custom code to Cloud Storage

Before you can deploy your model for serving, Vertex AI needs access to the following files in Cloud Storage:

* `model.joblib` (model artifact)
* `preprocessor.pkl` (model artifact)

Run the following commands to upload your files:

In [ ]:
%cd ..
!gsutil cp {LOCAL_MODEL_ARTIFACTS_DIR}/* {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
!gsutil ls {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

## Build a custom serving container using the CPR model server

Now that the model and processor has been trained and saved, it's time to build the custom serving container. Typically building a serving container requires writing model server code. However, with the Custom Prediction Routine feature, Vertex AI Prediction provides a model server that can be used out of the box.

A custom serving container contains the following 3 pieces of code:
1. [Model Server](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/model_server.py)
    * HTTP server that hosts the model.
    * Responsible for setting up routes/ports/etc.
1. [Request Handler](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/handler.py)
    * Responsible for webserver aspects of handling a request, such as deserializing the request body, serializing the response, setting response headers, etc.
    * In this example, you will use the default Handler, `google.cloud.aiplatform.prediction.handler.PredictionHandler` provided in the SDK.
1. [Predictor](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/predictor.py)
    * Responsible for the ML logic for processing a prediction request.

Each of these three pieces can be customized based on the requirements of the custom container. In this example, you will only be implementing the `Predictor`.


A [`Predictor`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/predictor.py) must implement the following interface:

```
from abc import ABC, abstractmethod
from typing import Any


class Predictor(ABC):
    """Interface of the Predictor class for Custom Prediction Routines.
    The Predictor is responsible for the ML logic for processing a prediction request.
    Specifically, the Predictor must define:
    (1) How to load all model artifacts used during prediction into memory.
    (2) The logic that should be executed at predict time.
    When using the default PredictionHandler, the Predictor will be invoked as follows:
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """

    def __init__(self):
        return

    @abstractmethod
    def load(self, artifacts_uri: str) -> None:
        """Loads the model artifact.
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
        """
        pass

    def preprocess(self, prediction_input: Any) -> Any:
        """Preprocesses the prediction input before doing the prediction.
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.
        """
        return prediction_input

    @abstractmethod
    def predict(self, instances: Any) -> Any:
        """Performs prediction.
        Args:
            instances (Any):
                Required. The instance(s) used for performing prediction.
        Returns:
            Prediction results.
        """
        pass

    def postprocess(self, prediction_results: Any) -> Any:
        """Postprocesses the prediction results.
        Args:
            prediction_results (Any):
                Required. The prediction results.
        Returns:
            The postprocessed prediction results.
        """
        return prediction_results
```

First, implement a custom `Predictor` that loads in both the preprocesor and the model. The preprocessor and the model will then be used at `predict` time.

Custom Prediction Routine supports a way to run the containers locally for testing your images. You can pass either a GCS path or a local path while testing your images locally.
- You need to set up the credentials if you pass a GCS path. 
- You need to support loading your models remotely and locally in your `Predictor` if you want to testing by passing a local path.

Vertex AI SDK provides a [function `download_model_artifacts`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/utils/prediction_utils.py) to help you download model artifacts from either GCS paths or local paths. See the example in the `load` function below.

In [ ]:
%%writefile $USER_SRC_DIR/predictor.py

import joblib
import numpy as np
import pickle

from google.cloud.aiplatform.prediction.predictor import Predictor
from google.cloud.aiplatform.utils import prediction_utils

from sklearn.datasets import load_iris


class CprPredictor(Predictor):
    
    def __init__(self):
        return
    
    def load(self, artifacts_uri: str):
        """Loads the preprocessor and model artifacts."""
        prediction_utils.download_model_artifacts(artifacts_uri)

        with open("preprocessor.pkl", "rb") as f:
            preprocessor = pickle.load(f)

        self._class_names = load_iris().target_names
        self._model = joblib.load("model.joblib")
        self._preprocessor = preprocessor

    def predict(self, instances):
        """Performs prediction."""
        instances = instances["instances"]
        inputs = np.asarray(instances)
        preprocessed_inputs = self._preprocessor.preprocess(inputs)
        outputs = self._model.predict(preprocessed_inputs)

        return {"predictions": [self._class_names[class_num] for class_num in outputs]}

To build a custom container, you also need to write an entrypoint of the image that starts the model server. However, with the Custom Prediction Routine feature, you don't need to write the entrypoint anymore. Vertex AI SDK will populate the entrypoint with the custom predictor you provide.

Write the dependencies to the source directory which will be installed in the image.

In [ ]:
!cp requirements.txt $USER_SRC_DIR/requirements.txt

## Build and push container to Artifact Registry

### Build your custom container

To build a custom image, a Dockerfile is necessary where you need to implement what the image looks like. However, with the Custom Prediction Routine features, Vertex AI SDK also generates Dockerfile and build images for you.

Using `python:3.7` as a base image by default.

In [ ]:
import os

from google.cloud.aiplatform.prediction import LocalModel
from src_dir_sdk.predictor import \
    CprPredictor  # Update this path as the variable $USER_SRC_DIR to import the custom predictor.

local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=CprPredictor,
    requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
)

You can check out the serving container spec of the built image.

In [ ]:
local_model.get_serving_container_spec()

### Store test instances

To learn more about formatting input instances in JSON, [read the documentation.](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
INPUT_FILE = "instances.json"

In [ ]:
%%writefile $INPUT_FILE
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

### Run and test the container locally

Custom Prediction Routines provide two ways to download model artifacts for testing images locally.
1. A local path.
2. A GCS path.

To use a local path, your `Predictor` needs to support both ways to load models so it can be tested locally and deployed to Vertex AI Prediction service. SDK provides a method `download_model_artifacts` to support these two ways in [prediction_utils.py](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/utils/prediction_utils.py) which you can call in the `load` function in your `Predictor`.

If you want to test images locally with a GCS path, you need to follow the instructions to set up the credentials.

#### Set up credentials

Setting up credentials is only required to run the custom serving container locally with GCS paths. Credentials set up is required to execute the `Predictor`'s `load` function, which downloads the model artifacts from Google Cloud Storage.

To access Google Cloud Storage in your project, you'll need to set up credentials by using one of the following:

1. User account

1. Service account

You can learn more about each of the above [here](https://cloud.google.com/docs/authentication#principals).

**Option 1. Use Google user credentials**

First authorize Google auth library to access the Cloud Platform with Google user credentials. This step involves an interactive prompt which requires user inputs. If you are using a non-interactive shell, you should open a terminal to run this command. See [here](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html) for how to open a terminal in a Vertex Workbench notebook environment.

Note that if you are running the command below on a machine without a window manager/browser, it will prompt you to run the command `gcloud auth application-default login --remote-bootstrap="..."`. Only machines that can launch web browsers, e.g. laptops or workstations, are allowed to run this command. After completing the authentication steps using your browser, you'll need to copy and paste the verification code back into the original terminal to continue the login flow. If you are running this command on a machine with web browser support, there is no need to use a separate machine.

This command will print the credential file as `Credentials saved to file: [CREDENTIALS_FILE]`.

In [ ]:
!gcloud auth application-default login

Specify the credential file, which is a json file, as the path.

In [ ]:
CREDENTIALS_FILE = "[CREDENTIALS_FILE]"  # @param {type:"string"}

Next, authorize gcloud to access the Cloud Platform with Google user credentials. The user credentials need to have `setIamPolicy` permission in the project. See [here](https://cloud.google.com/resource-manager/docs/access-control-proj) for more details.

Similar to the previous step, this also requires user inputs. Instructions for the previous step apply here as well.

In [ ]:
!gcloud auth login

Grant roles to the user account. Use the email address that you used in the gcloud auth step, e.g., `myemail@gmail.com`. Since you will be using this user account to download the model artifacts from Google Cloud Storage, you grant the user account **Storage Object Viewer** role. See [IAM roles for Cloud Storage](https://cloud.google.com/storage/docs/access-control/iam-roles) for more details.

In [ ]:
USER_ACCOUNT = "[USER_ACCOUNT]"  # @param {type:"string"}

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=user:$USER_ACCOUNT --role=roles/storage.objectViewer

Initialize the Vertex AI SDK with the project ID, which is required for using user credentials.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

**Option 2. Use Google Service Account credentials**

Skip this if you've already completed Option 1 above. First enable the IAM API if it's not already enabled.

In [ ]:
!gcloud services enable iam.googleapis.com

#### Service Account

You use a service account to download model artifacts while you run containers locally. If you do not want to use your project's Compute Engine service account, set `SERVICE_ACCOUNT` to another service account ID.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

Create the service account.

In [ ]:
!gcloud iam service-accounts create $SERVICE_ACCOUNT

Authorize gcloud to access the Cloud Platform with Google user credentials. The user credentials need to have `setIamPolicy` permission in the project. See [here](https://cloud.google.com/resource-manager/docs/access-control-proj) for more details.

This step involves an interactive prompt which requires user inputs. If you are using a non-interactive shell, you should open a terminal to run this command. See [here](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html) for how to open a terminal in a Vertex Workbench notebook environment.

Note that if you are running the command below on a machine without a window manager/browser, it will prompt you to run the command `gcloud auth application-default login --remote-bootstrap="..."`. Only machines that can launch web browsers, e.g. laptops or workstations, are allowed to run this command. After completing the authentication steps using your browser, you'll need to copy and paste the verification code back into the original terminal to continue the login flow. If you are running this command on a machine with web browser support, there is no need to use a separate machine.

In [ ]:
!gcloud auth login

Grant roles to the service account. Since you will be using this service account to download the model artifacts from Google Cloud Storage, yiy grant the service account **Storage Object Viewer** role. See [IAM roles for Cloud Storage](https://cloud.google.com/storage/docs/access-control/iam-roles) for more details.

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com \
    --role=roles/storage.objectViewer

Specify the service account key file name.

In [ ]:
CREDENTIALS_FILE = "./credentials.json"

Generate the service account key file. This will be used while running the custom serving container locally.

In [ ]:
!gcloud iam service-accounts keys create $CREDENTIALS_FILE \
    --iam-account="{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com"

#### Run and send requests to the container locally

With the Custom Prediction Routine feature, it is easy to test the container locally.

In this example, the container executes a prediction request and a health check.

**Option 1. Pass local paths**

This option requires the `load` function in the `Predictor` supports loading model artifacts from both of GCS paths and local paths.

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{LOCAL_MODEL_ARTIFACTS_DIR}",
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

Print out the predict response and its content.

In [ ]:
print(predict_response, predict_response.content)

Print out the health check response and its content.

In [ ]:
print(health_check_response, health_check_response.content)

Also print out all the container logs. You will see the logs of container startup, serving requests, and container teardown.

In [ ]:
local_endpoint.print_container_logs(show_all=True)

**Option 2. Pass GCS paths**

If you want to pass GCS paths, you need to have the credentials set up in the previous step and pass the path to the credentials while running the container. The service account should have the **Storage Object Viewer** permission.

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    credential_path=CREDENTIALS_FILE,
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

Print out the predict response and its content.

In [ ]:
print(predict_response, predict_response.content)

Print out the health check response and its content.

In [ ]:
print(health_check_response, health_check_response.content)

Also print out all the container logs. You will see the logs of container startup, serving requests, and container teardown.

In [ ]:
local_endpoint.print_container_logs(show_all=True)

### Push the container to artifact registry

Configure Docker to access Artifact Registry. Then push your container image to your Artifact Registry repository.

Print out all enabled services in your project.

In [ ]:
!gcloud services list

If `artifactregistry.googleapis.com` is not enabled in your project, enable the API before proceeding.

In [ ]:
!gcloud services enable artifactregistry.googleapis.com

In [ ]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

Use SDK to push the image.

In [ ]:
local_model.push_image()

## Deploy to Vertex AI

### Upload the custom container model

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

Use the LocalModel instance to upload the model. It will populate the container spec automatically for you.

In [ ]:
model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
)

### Deploy the model on Vertex AI
After this step completes, the model is deployed and ready for online prediction.

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

## Send predictions

### Using Python SDK

In [ ]:
endpoint.predict(instances=[[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]])

### Using REST

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

### Using gcloud CLI

In [ ]:
!gcloud ai endpoints predict $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances.json

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI